# Vietnam



## Table of contents
1. [General Geography](#1)<br>
    1.1 [Soil Map](#11)<br>
    1.2 [Road and Railway Network](#12)<br>
2. [Poverty in Vietnam](#2)<br>
    2.1 [The percentage of malnourished children under 5 in 2018 by locality](#21)<br>
    2.2 [Proportion of poor households by region in Vietnam from 1998 to 2016 ](#22)<br>
3. [Vietnam Economy](#3)<br>
    3.1 [Employment](#31)<br>
    3.2 [The Aquaculture Production of Vietnam from 2013 to 2018 by Provinces](#32)<br>
    3.3 [Various sources of income by provinces  in 2018 in Vietnam](#33)<br>

I. [Important Notes](#333)<br>
II. [References](#666)<br>

In [1]:
import json
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd
import shapely.geometry
from ipywidgets import widgets
# Plot in browser
import plotly.io as pio
pio.renderers.default = 'browser'

from codes.auxiliary import convert_id_map
from codes.plot import *

## 1. General Geography <a id="1"></a>

### 1.1 Soil Map <a id="11"></a>

Dataset of soil types of Vietnam is a geospatial polygon data which is based on FAO classification

[Source](https://data.opendevelopmentmekong.net/dataset/soil-types-in-vietnam)

In [2]:
soil_geo = json.load(open('geodata/soilmap_vietnam.geojson',"r"))

# split unique soil type
imap = convert_id_map(soil_geo, 'Type', 'faosoil')
map_keys = imap.keys()
soil_list = []
soil_dict = {}
for key in map_keys:
    soil_type = key.split("sols")[0]
    soil_type +="sols"
    if key not in soil_dict.keys():
        soil_dict[key] = soil_type
    if soil_type not in soil_list:
        soil_list.append(soil_type)

# Soilmap Dataframe
soil_pd = gpd.read_file('geodata/soilmap_vietnam.geojson')
# soil_pd = soil_pd.iloc[:,0:4]
soil_pd["Soil_type"] = soil_pd['Type'].map(soil_dict)

In [3]:
# Plotting soil map
fig = px.choropleth_mapbox(
    soil_pd,
    geojson=soil_geo,
    color = "Soil_type",
    color_discrete_sequence=px.colors.qualitative.Light24,
    locations = "gid",
    featureidkey="properties.gid",
    mapbox_style = "carto-positron",
    center = {"lat": 16, "lon": 106},
    zoom = 5,
    title = "Soil Map"
    )
    
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

<img src="figures/soilmap.png" alt="drawing" style="width:950px;"/>

### 1.2 Road and Railway network <a id="12"></a>

A geospatial dataset containing polylines of transportation network in Vietnam. It contains the railways, the principal roads and the secondary roads.
[Source](https://data.opendevelopmentmekong.net/dataset/giao-thng-vit-nam)

In [4]:
# open a zipped shapefile with the zip:// pseudo-protocol
transport_df = gpd.read_file("geodata/transport.zip")

lats = []
lons = []
names = []

for feature, name in zip(transport_df.geometry, transport_df.Name):
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        names = np.append(names, [name]*len(y))
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        names = np.append(names, None)

fig = px.line_mapbox(lat=lats, lon=lons, hover_name=names,
                     mapbox_style="stamen-terrain", zoom=4.5, center={"lat": 16, "lon": 106})

fig.show()

<img src="figures/South_railway_road.png" alt="drawing" style="width:950px;"/>

## 2. Poverty In Vietnam <a id="2"></a>

### 2.1. The percentage of malnourished children under 5 in 2018 by locality <a id="21"></a>

The attributes include the total weight, hight and weight based on height.

In [6]:
# Malnutrition data
malnutrition_children_vn_2018  = pd.read_csv("geodata/malnutrition_children_vn_2018.csv")

#Vietnam map
vietnam_geo = json.load(open("geodata/vietnam_state.geojson","r"))

# Plotting
fig = px.choropleth_mapbox(
    malnutrition_children_vn_2018,
    locations = 'Code',
    featureidkey="properties.Code",
    geojson = vietnam_geo,
    color = 'Wei_Hei',
    hover_name = "Name",
    mapbox_style = "carto-positron",
    center = {"lat": 16,"lon": 106},
    zoom = 4.5,
    title = "malnourished children under 5 in 2018 by locality in Vietnam ",
)
fig.update_geos(fitbounds = "locations", visible=False)
fig.show()

<img src="figures/Malnutrion_children_2018.png" alt="drawing" style="width:950px;"/>

### 2.2 Proportion of poor households by region in Vietnam from 1998 to 2016  <a id="22"></a>
The dataset includes le pourcentage of poor households by region in Vietnam from 1998 to 2016. The standard of poor households for this period based on the average income per person per month of households is updated according to the consumer price index as follows: In 2010, VND 400,000 for rural areas and VND 500,000 for urban areas; Similarly, in 2013 it was VND 570,000 and VND 710,000; in 2014, VND 605,000 dong and VND 750,000; in 2015, there were VND 615,000 and VND 760,000 dong; In 2016, VND 630,000 and VND 780,000 respectively.

In [7]:
# Import the Vietnam map by region data (error geojson file)
vnregion_geo = json.load(open("geodata/poverty_rate_1998_2016.geojson", "r",encoding='utf-8'))

# Import aquaculture_production csv
poverty_rate_1998_2016 = pd.read_csv("geodata/poverty_rate_1998_2016.csv")
cols = sorted(poverty_rate_1998_2016.columns[3:], reverse=False)

for i, y in enumerate(cols):
    poverty = "Poverty" + y
    poverty_rate_1998_2016[poverty] = poverty_rate_1998_2016[y]

# Convert wide to long format
poverty = poverty_rate_1998_2016.drop(cols, axis=1)
final_poverty = pd.wide_to_long(poverty,"Poverty", i=['Name_EN','Name_VI','id'], j= "year")
final_poverty.reset_index(inplace=True)

### Choropleth map using GeoJSON

In [8]:
input_year ='1998' #1998 2002 2004 2006 2008 2010 2011 2012 2013 2014 2015 2016

fig = px.choropleth_mapbox(
    poverty_rate_1998_2016,
    locations = 'id',
    geojson = vnregion_geo,
    featureidkey="properties.id",
    color = "Poverty" + input_year ,
    color_continuous_scale="Viridis",
    range_color=(0, 65),
    hover_name = "Name_EN",
    # hover_data = ["Poverty_percentage" + input_year],
    mapbox_style = "carto-positron",
    center = {"lat": 17,"lon": 106},
    zoom = 4.5,
    title = "Proportion of poor households by region in Vietnam "+ input_year,
)
fig.update_geos(fitbounds = "locations", visible=False)
fig.show()

<img src="figures/poverty_rate_1998_2016.png" alt="drawing" style="width:950px;"/>

### Animated figures with GeoJSON, Plotly Express

In [9]:
fig = px.choropleth(
    final_poverty,
    locations = 'id',
    featureidkey="properties.id",
    animation_frame = "year",
    geojson = vnregion_geo,
    color = "Poverty",
    color_continuous_scale="Viridis",
    range_color=(0, 65),
    hover_name = "Name_EN",
    # hover_data = ['Poverty_percentage'],
    title = "Proportion of poor households by region in Vietnam from 1998 to 2016",
)
fig.update_geos(fitbounds = "locations", visible=False)
fig.show()

## 3. Vietnam Economy <a id="3"></a>

### 3.1 Employment <a id="31"></a>

[Source](https://www.gso.gov.vn/en/employment/)


In [10]:
# Import csv
trained_employee = pd.read_csv("geodata/trained_employee15_vn.csv")
labor_force = pd.read_csv("geodata/labor_force_vn.csv")

<img src="figures/Percent_employ15.png" alt="drawing" style="width:950px;"/>

#### Labour force at 15 years of age and above by province 

In [11]:
title31 = "Labour force at 15 years of age and above by province from 2010 to 2018"
plot_animation_frame_vietnamstate(labor_force,vietnam_geo,"labor_force", title31)

#### Percentage of employed workers at 15 years of age and above among population by province 

In [12]:
title32 = "Percentage of employed workers at 15 years of age and above among population by province from 2010 to 2018"
plot_animation_frame_vietnamstate(trained_employee,vietnam_geo,"percentage", title32)


### 3.2 The Aquaculture Production of Vietnam from 2013 to 2018 by Provinces <a id="32"></a>
Published by: Open Development Vietnam The data provides information on Vietnam's aquaculture production from 2013 to 2018. The aquaculture in Vietnam includes: farmed fish production, farmed shrimp production and other aquatic products. Aquaculture production is divided by province and city.

In [13]:
# import the Vietnam map by provinces data
vietnam_geo = json.load(open("geodata/vietnam_state.geojson", "r"))

# Convert map properties/
state_id_map = convert_id_map(vietnam_geo, "Name", "Code")

In [14]:
# Import aquaculture_production csv
df = pd.read_csv("geodata/aquaculture_production_2013__2018.csv")
years = ['2013','2014','2015','2016','2017','2018']
for i, y in enumerate(years):
    scale = 'Production_Scale'+ y
    prod = 'Production' + y
    df[scale] = np.log10(df[y])
    df[prod] = df[y]

# Convert wide to long format
prod = df.drop(years, axis=1)
final_prod = pd.wide_to_long(prod, stubnames=['Production_Scale','Production'], i=['Name','Code'], j="year")
final_prod.reset_index(inplace=True)

### Choropleth map using GeoJSON

In [15]:
input_year = '2018'
fig = px.choropleth_mapbox(
    df,
    geojson = vietnam_geo,
    locations ="Code",
    color = "Production_Scale" + input_year,
    range_color=(2, 6),
    hover_name = "Name",
    featureidkey = "properties.Code",
    hover_data = ['Production'+ input_year],
    mapbox_style="carto-positron",
    center={"lat": 16, "lon": 106},
    zoom=4.5,
    title ="The Aquaculture Production of Vietnam by Province in " + input_year
)
fig.update_geos(fitbounds ="locations", visible=False)
fig.show()

<img src="figures/Aqua_prod_2013.png" alt="drawing" style="width:950px;"/>

### Animated figures with GeoJSON, Plotly Express

In [16]:
title33 = "The Aquaculture Production of Vietnam from 2013 to 2018 by Province"
plot_animation_frame_vietnamstate(final_prod, vietnam_geo, "Production_Scale", title33)

## 3.3 Various sources of income by provinces  in 2018 in Vietnam  <a id="33"></a>

The data provide the information on per capita in come by province in 2018 in Vietnam. The total income per month includes the salary, the income from agriculture, forestry, aquaculture and non-agriculture, non-forestry, non-aquaculture and from other income. The income unit is in thousands vnd

[Source](https://data.opendevelopmentmekong.net/dataset/per-capita-income-by-province-in-2018-in-vietnam)

In [17]:
# Import csv and geojson
income_df = pd.read_csv("geodata/thunhapbinhquan.csv")
categories = sorted(income_df.columns[3:])

#Vietnam map
vietnam_geo = json.load(open("geodata/vietnam_state.geojson","r"))

<img src="figures/Wage_agri_by_province.png" alt="drawing" style="width:950px;"/>

In [18]:
trace = go.Choroplethmapbox(
    geojson = vietnam_geo,
    featureidkey='properties.Code',
    locations = income_df["Code"],
    z=income_df.loc[0:, 'income_total_average'],
    hovertext = 'Province: ' + income_df.Name_EN,
    colorscale ='viridis',
    marker_opacity=0.9,
    marker_line_width=0.9,
    showscale=True
    )

lyt = dict(title='Income by provinces',
           height = 700,
           mapbox_style = "white-bg",
           mapbox_zoom = 4,
           mapbox_center = {"lat": 17,"lon": 106})

fig = go.FigureWidget(data=[trace], layout=lyt)

# Add dropdowns
## 'Income' dropdown
cat_options = ['total_average', 'salary', 'agri', 'non_agri', 'others']
category = widgets.Dropdown(options=cat_options,
                            value='total_average',
                            description='Category')

# Add Submit button
submit = widgets.Button(description='Submit',
                        disabled=False,
                        button_style='info',
                        icon='check')

In [19]:
def submit_event_handler(args):
    if category.value in cat_options:
        new_data = income_df.loc[0:, 'income_' + str(category.value)]
        with fig.batch_update():
            fig.data[0].z = new_data
            fig.layout.title = ' '.join(['Income ',str(category.value), ' in 2018'])

submit.on_click(submit_event_handler)

container = widgets.HBox([category, submit])
widgets.VBox([container, fig])

## Important Notes <a id="333"></a>


## Reference <a id="666"></a>
For additional information and attributes for creating bubble charts in Plotly see: https://plotly.com/python/bubble-charts/.

For more documentation on creating animations with Plotly, see https://plotly.com/python/animations.